# Building Optimization

This notebook performs building design optimization using EnergyPlus and BESOS helper functions.
We load a model from in.idf, define parameters to vary, set objectives, test the model, then run a multi-objective genetic algorithm and plot the optimized designs.

### Import libraries

In [ ]:
import pandas as pd
from besos import eppy_funcs as ef, sampling
from besos.evaluator import EvaluatorEP
from besos.optimizer import NSGAII, df_solution_to_solutions
from besos.parameters import RangeParameter, expand_plist, wwr
from besos.problem import EPProblem
from matplotlib import pyplot as plt
from platypus import Archive, Hypervolume, Solution

### Load the base EnergyPlus .idf file

In [ ]:
building = ef.get_building("in.idf")

### Define design parameters and ranges
Define a parameter list using a helper function, in this case building orientation and window-to-wall ratio.


In [ ]:
parameters = []
parameters = expand_plist(
    {"Building 1": {"North Axis": (0, 359)}}  # Name from IDF Building object
)

parameters.append(
    wwr(RangeParameter(0.1, 0.9))
)  # Add window-to-wall ratio as a parameter between 0.1 and 0.9 using a custom function

### Objectives
Using Heating and Cooling energy outputs as simulation objectives, make a problem instance from these parameters and objectives.

In [ ]:
objectives = ["DistrictCooling:Facility", "DistrictHeating:Facility"]
besos_problem = EPProblem(parameters, objectives)

### Set up EnergyPlus evaluator object to run simulations for this building and problem

In [ ]:
evaluator = EvaluatorEP(
    besos_problem, building, out_dir="outputdir", err_dir="outputdir"
)  # outputdir must exist; E+ files will be written there
runs = pd.DataFrame.from_dict(
    {"0": [180, 0.5]}, orient="index"
)  # Make a dataframe of runs with one entry for South and 50% glazing
outputs = evaluator.df_apply(runs)  # Run this as a test
outputs

## Run the Genetic Algorithm
Run the optimizer using this evaluator for a population size of 20 for 10 generations.

In [ ]:
results = NSGAII(evaluator, evaluations=10, population_size=20)
results

### Visualize the results

In [ ]:
optres = results.loc[
    results["pareto-optimal"] == True, :
]  # Get only the optimal results
plt.plot(
    results["DistrictCooling:Facility"], results["DistrictHeating:Facility"], "x"
)  # Plot all results in the background as blue crosses
plt.plot(
    optres["DistrictCooling:Facility"], optres["DistrictHeating:Facility"], "ro"
)  # Plot optimal results in red
plt.xlabel("Cooling demand")
plt.ylabel("Heating demand")

In [ ]:
optres = optres.sort_values("DistrictCooling:Facility")  # Sort by the first objective
optresplot = optres.drop(columns="violation")  # Remove the constraint violation column
ax = optresplot.plot.bar(
    subplots=True, legend=None, figsize=(10, 10)
)  # Plot the variable values of each of the optimal solutions

### Hypervolume

Now that initial results have been produced and verified against expectations, enlarge evaluations and population size to produce increased optimization of results.

In [ ]:
results_2 = NSGAII(evaluator, evaluations=20, population_size=50)

### Compare first run and second run

In [ ]:
optres_2 = results_2.loc[results_2["pareto-optimal"] == True, :]
plt.plot(
    optres["DistrictCooling:Facility"], optres["DistrictHeating:Facility"], "bo"
)  # Plot first optimal results in blue
plt.plot(
    optres_2["DistrictCooling:Facility"], optres_2["DistrictHeating:Facility"], "ro"
)  # Plot second optimal results in red
plt.xlabel("Cooling demand")
plt.ylabel("Heating demand")

### Calculate the hypervolume

In [ ]:
reference_set = Archive()
platypus_problem = evaluator.to_platypus()
for _ in range(20):
    solution = Solution(platypus_problem)
    solution.variables = sampling.dist_sampler(
        sampling.lhs, besos_problem, 1
    ).values.tolist()[0]
    solution.evaluate()
    reference_set.add(solution)

hyp = Hypervolume(reference_set)
print(
    "Hypervolume for result 1:",
    hyp.calculate(df_solution_to_solutions(results, platypus_problem, besos_problem)),
)
print(
    "Hypervolume for result 2:",
    hyp.calculate(df_solution_to_solutions(results_2, platypus_problem, besos_problem)),
)